# Banking AI Agent with LangGraph - Comprehensive Demo

This notebook demonstrates how to build and validate a **banking-specific AI agent** using LangGraph integrated with ValidMind for comprehensive testing and monitoring.

## What You'll Learn

Learn how to create intelligent banking agents that can:
- **Automatically select appropriate banking tools** based on user queries using LLM-powered routing
- **Handle regulatory compliance** with built-in SR 11-7 and SS 1-23 monitoring
- **Manage credit risk assessment** for loan applications and decisions
- **Provide financial calculations** for loans, investments, and account analysis
- **Monitor fraud and security** with comprehensive risk scoring
- **Deliver customer service** through account management and product recommendations

## Target Use Cases

This agent is designed for:
- **Retail Banking**: Personal loans, account management, customer service
- **Commercial Banking**: Business loans, large transactions, compliance monitoring
- **Risk Management**: Credit analysis, fraud detection, regulatory compliance
- **Customer Service**: Account inquiries, product recommendations, transaction support

We'll build a comprehensive banking agent system that intelligently routes user requests to five specialized tools, then validate its performance using ValidMind's testing framework.


## Setup and Imports

First, let's import all the necessary libraries for building our banking LangGraph agent system:

- **LangChain components** for LLM integration and tool management
- **LangGraph** for building stateful, multi-step agent workflows
- **ValidMind** for model validation and testing
- **Banking tools** for specialized financial services
- **Standard libraries** for data handling and environment management

The setup includes loading environment variables (like OpenAI API keys) needed for the LLM components to function properly.


In [1]:
%pip install -q langgraph langchain validmind openai


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import TypedDict, Annotated, Sequence, Optional
from langchain.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
import pandas as pd
from datetime import datetime

# Load environment variables if using .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("dotenv not installed. Make sure OPENAI_API_KEY is set in your environment.")


In [ ]:
import validmind as vm

vm.init(
    api_host="...",
    api_key="...",
    api_secret="...",
    model="...",
)

2025-09-03 17:29:08,173 - INFO(validmind.api_client): 🎉 Connected to ValidMind!
📊 Model: [Demo] Customer Churn Model (ID: clul6y51o02ct1ojrog2d4dus)
📁 Document Type: model_documentation


## Banking Tools Import

Now let's import our specialized banking tools that provide comprehensive financial services:

### Tool Overview
1. **Credit Risk Analyzer** - Loan applications and credit decisions
2. **Compliance Monitor** - SR 11-7, SS 1-23, and KYC/AML compliance
3. **Financial Calculator** - Banking calculations and analysis
4. **Customer Account Manager** - Account services and customer support
5. **Fraud Detection System** - Security and fraud prevention


In [4]:
# Import our specialized banking tools
from banking_tools import AVAILABLE_TOOLS

print("Banking Tools Successfully Loaded!")
print(f"Available tools: {len(AVAILABLE_TOOLS)}")
print("\nTool Details:")
for i, tool in enumerate(AVAILABLE_TOOLS, 1):
    print(f"   - {tool.name}: {tool.description[:50]}...")
    
print("All banking tools are ready for use!")

# For testing purposes, we can access the underlying functions using .func
# This is useful for debugging and testing individual tools
print("\nNote: For direct testing, use tool.func() to access underlying functions")
print("   Example: credit_risk_analyzer.func(customer_income=75000, ...)")


Banking Tools Successfully Loaded!
Available tools: 5

Tool Details:
   - credit_risk_analyzer: Analyze credit risk for loan applications and cred...
   - compliance_monitor: Monitor transactions and operations for regulatory...
   - financial_calculator: Perform banking-specific financial calculations.

...
   - customer_account_manager: Manage customer accounts and provide banking servi...
   - fraud_detection_system: Analyze transactions for potential fraud and secur...
All banking tools are ready for use!

Note: For direct testing, use tool.func() to access underlying functions
   Example: credit_risk_analyzer.func(customer_income=75000, ...)


## Test Banking Tools Individually

Let's test each banking tool individually to ensure they're working correctly before integrating them into our agent.

**Important Note**: Since these are LangChain tools, we need to use `.func` to access the underlying functions for direct testing.


In [5]:
print("Testing Individual Banking Tools\n")
print("=" * 60)

# Test 1: Credit Risk Analyzer
print("\nTEST 1: Credit Risk Analyzer")
print("-" * 40)
try:
    # Access the underlying function using .func
    credit_result = AVAILABLE_TOOLS[0].func(
        customer_income=75000,
        customer_debt=1200,
        credit_score=720,
        loan_amount=50000,
        loan_type="personal"
    )
    print(credit_result)
    print("Credit Risk Analyzer test PASSED")
except Exception as e:
    print(f"Credit Risk Analyzer test FAILED: {e}")

print("\n" + "=" * 60)

# Test 2: Compliance Monitor
print("\nTEST 2: Compliance Monitor (SR 11-7)")
print("-" * 40)
try:
    compliance_result = AVAILABLE_TOOLS[1].func(
        transaction_type="loan",
        transaction_amount=150000,
        customer_type="commercial",
        account_age_days=90,
        compliance_framework="SR_11_7"
    )
    print(compliance_result)
    print("Compliance Monitor test PASSED")
except Exception as e:
    print(f"Compliance Monitor test FAILED: {e}")

print("\n" + "=" * 60)

# Test 3: Financial Calculator
print("\nTEST 3: Financial Calculator")
print("-" * 40)
try:
    calc_result = AVAILABLE_TOOLS[2].func(
        calculation_type="loan_payment",
        principal=300000,
        rate=4.5,
        term=30,
        payment_frequency="monthly"
    )
    print(calc_result)
    print("Financial Calculator test PASSED")
except Exception as e:
    print(f"Financial Calculator test FAILED: {e}")

print("\nAll individual tool tests completed!")
print("\nNote: These tools are now ready for the LangGraph agent to use automatically!")


Testing Individual Banking Tools


TEST 1: Credit Risk Analyzer
----------------------------------------
CREDIT RISK ANALYSIS REPORT
    
    Customer Profile:
    - Annual Income: $75,000.00
    - Monthly Debt: $1,200.00
    - Credit Score: 720
    - Loan Request: $50,000.00 (personal)
    
    Risk Assessment:
    - Debt-to-Income Ratio: 19.2%
    - Risk Score: 70/75
    - Risk Level: LOW RISK
    
    Recommendation: APPROVE with standard terms
    
    Additional Notes:
    - DTI ratio of 19.2% is excellent
    - Credit score of 720 is good
    - Loan amount represents 66.7% of annual income
    
Credit Risk Analyzer test PASSED


TEST 2: Compliance Monitor (SR 11-7)
----------------------------------------
COMPLIANCE MONITORING REPORT
    
    Transaction Details:
    - Type: Loan
    - Amount: $150,000.00
    - Customer Type: Commercial
    - Account Age: 90 days
    - Framework: Sr 11 7
    
    Compliance Assessment:
    - Risk Level: MEDIUM
    - Issues Found: 2
    
    Compl

## Complete LangGraph Banking Agent

Now we'll create our intelligent banking agent with LangGraph that can automatically select and use the appropriate banking tools based on user requests.


In [ ]:
# Banking Agent State Definition
class BankingAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    user_input: str
    session_id: str
    context: dict

def create_banking_langgraph_agent():
    """Create a comprehensive LangGraph banking agent with intelligent tool selection."""
    
    # Initialize the main LLM for banking responses
    main_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    # Bind all banking tools to the main LLM
    llm_with_tools = main_llm.bind_tools(AVAILABLE_TOOLS)
    
    def llm_node(state: BankingAgentState) -> BankingAgentState:
        """Main LLM node that processes banking requests and selects appropriate tools."""
        
        messages = state["messages"]
        
        # Enhanced banking system prompt with tool selection guidance
        system_context = """You are a professional banking AI assistant with access to specialized banking tools.
            Analyze the user's banking request and directly use the most appropriate tools to help them.
            
            AVAILABLE BANKING TOOLS:
            
            credit_risk_analyzer - Analyze credit risk for loan applications and credit decisions
            - Use for: loan applications, credit assessments, risk analysis, mortgage eligibility
            - Examples: "Analyze credit risk for $50k personal loan", "Assess mortgage eligibility for $300k home purchase"
            - Parameters: customer_income, customer_debt, credit_score, loan_amount, loan_type

            compliance_monitor - Monitor transactions and operations for regulatory compliance
            - Use for: SR 11-7, SS 1-23 compliance, KYC/AML requirements, regulatory reporting
            - Examples: "Check SR 11-7 compliance for $100k business loan", "Verify AML compliance for $25k wire transfer"
            - Parameters: transaction_type, transaction_amount, customer_type, account_age_days, compliance_framework

            financial_calculator - Perform banking-specific financial calculations
            - Use for: loan payments, interest calculations, investment returns, amortization schedules
            - Examples: "Calculate monthly payment for $200k mortgage at 4.5% for 30 years", "Compute investment growth for $50k at 8% return"
            - Parameters: calculation_type, principal, rate, term, payment_frequency

            customer_account_manager - Manage customer accounts and provide banking services
            - Use for: account information, transaction processing, product recommendations, customer service
            - Examples: "Check balance for checking account 12345", "Recommend products for customer with high balance"
            - Parameters: account_type, customer_id, action, amount, account_details

            fraud_detection_system - Analyze transactions for potential fraud and security risks
            - Use for: transaction monitoring, fraud prevention, risk assessment, security alerts
            - Examples: "Analyze fraud risk for $500 ATM withdrawal in Miami", "Check security for $2000 online purchase"
            - Parameters: transaction_id, customer_id, transaction_amount, transaction_type, location, device_id

            BANKING INSTRUCTIONS:
            - Analyze the user's banking request carefully and identify the primary need
            - If they need credit analysis → use credit_risk_analyzer
            - If they need compliance checks → use compliance_monitor
            - If they need financial calculations → use financial_calculator
            - If they need account services → use customer_account_manager
            - If they need security analysis → use fraud_detection_system
            - Extract relevant parameters from the user's request
            - Provide helpful, accurate banking responses based on tool outputs
            - Always consider banking regulations, risk management, and best practices
            - Be professional and thorough in your analysis

            Choose and use tools wisely to provide the most helpful banking assistance."""
        
        # Add system context to messages
        enhanced_messages = [SystemMessage(content=system_context)] + list(messages)
        
        # Get LLM response with tool selection
        response = llm_with_tools.invoke(enhanced_messages)
        
        return {
            **state,
            "messages": messages + [response]
        }
    
    def should_continue(state: BankingAgentState) -> str:
        """Decide whether to use tools or end the conversation."""
        last_message = state["messages"][-1]
        
        # Check if the LLM wants to use tools
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "tools"
        
        return END
        
    # Create the banking state graph
    workflow = StateGraph(BankingAgentState)
    
    # Add nodes
    workflow.add_node("llm", llm_node)
    workflow.add_node("tools", ToolNode(AVAILABLE_TOOLS))
    
    # Simplified entry point - go directly to LLM
    workflow.add_edge(START, "llm")
    
    # From LLM, decide whether to use tools or end
    workflow.add_conditional_edges(
        "llm",
        should_continue,
        {"tools": "tools", END: END}
    )
    
    # Tool execution flows back to LLM for final response
    workflow.add_edge("tools", "llm")
    
    # Set up memory
    memory = MemorySaver()
    
    # Compile the graph
    agent = workflow.compile(checkpointer=memory)
    
    return agent

# Create the banking intelligent agent
banking_agent = create_banking_langgraph_agent()

print("Banking LangGraph Agent Created Successfully!")
print("\nFeatures:")
print("   - Intelligent banking tool selection")
print("   - Comprehensive banking system prompt")
print("   - Streamlined workflow: LLM → Tools → Response")
print("   - Automatic tool parameter extraction")
print("   - Professional banking assistance")


Banking LangGraph Agent Created Successfully!

Features:
   - Intelligent banking tool selection
   - Comprehensive banking system prompt
   - Streamlined workflow: LLM → Tools → Response
   - Automatic tool parameter extraction
   - Professional banking assistance


## ValidMind Model Integration

Now we'll integrate our banking LangGraph agent with ValidMind for comprehensive testing and validation.


In [7]:
def banking_agent_fn(input):
    """
    Invoke the banking agent with the given input.
    """
    try:
        # Initial state for banking agent
        initial_state = {
            "user_input": input["input"],
            "messages": [HumanMessage(content=input["input"])],
            "session_id": input["session_id"],
            "context": {}
        }

        session_config = {"configurable": {"thread_id": input["session_id"]}}

        result = banking_agent.invoke(initial_state, config=session_config)

        return {"prediction": result['messages'][-1].content, "output": result}
    except Exception as e:
        # Return a fallback response if the agent fails
        error_message = f"I apologize, but I encountered an error while processing your banking request: {str(e)}. Please try rephrasing your question or contact support if the issue persists."
        return {
            "prediction": error_message, 
            "output": {
                "messages": [HumanMessage(content=input["input"]), SystemMessage(content=error_message)],
                "error": str(e)
            }
        }


vm_banking_model = vm.init_model(input_id="banking_agent_model", predict_fn=banking_agent_fn)
# Add the banking agent to the vm model
vm_banking_model.model = banking_agent

print("Banking Agent Successfully Integrated with ValidMind!")
print(f"Model ID: {vm_banking_model.input_id}")


Banking Agent Successfully Integrated with ValidMind!
Model ID: banking_agent_model


## Prepare Banking Test Dataset

We'll use our comprehensive banking test dataset to evaluate our agent's performance across different banking scenarios.


In [8]:
# Import our banking-specific test dataset
from banking_test_dataset import banking_test_dataset

# Use the banking dataset for comprehensive testing
test_dataset = banking_test_dataset

print("Banking Test Dataset Loaded Successfully!")
print(f"Number of test cases: {len(test_dataset)}")
print(f"Tools being tested: {sorted(test_dataset['expected_tools'].explode().unique())}")
print(f"Categories: {sorted(test_dataset['category'].unique())}")

print("\nTest Cases Preview:")
for i, row in test_dataset.iterrows():
    print(f"{i+1}. {row['input'][:80]}... -> Expected tool: {row['expected_tools'][0]} ({row['category']})")


Banking-specific test dataset created!
Number of test cases: 10
Categories: ['credit_risk' 'compliance' 'financial_calculation' 'account_management'
 'fraud_detection']
Tools being tested: ['compliance_monitor', 'credit_risk_analyzer', 'customer_account_manager', 'financial_calculator', 'fraud_detection_system']

Sample test cases:
1. Analyze credit risk for a $50,000 personal loan application with $75,000 annual ... -> Expected tool: credit_risk_analyzer (credit_risk)
2. Check SR 11-7 compliance for a $150,000 business loan to a commercial customer w... -> Expected tool: compliance_monitor (compliance)
3. Calculate monthly payment for a $300,000 mortgage at 4.5% interest for 30 years... -> Expected tool: financial_calculator (financial_calculation)
Banking Test Dataset Loaded Successfully!
Number of test cases: 10
Tools being tested: ['compliance_monitor', 'credit_risk_analyzer', 'customer_account_manager', 'financial_calculator', 'fraud_detection_system']
Categories: ['account_manage

### Initialize ValidMind Dataset

Before we can run tests and evaluations, we need to initialize our banking test dataset as a ValidMind dataset object.


In [9]:
vm_test_dataset = vm.init_dataset(
    input_id="banking_test_dataset",
    dataset=test_dataset,
    target_column="possible_outputs"
)

print("Banking Test Dataset Initialized in ValidMind!")
print(f"Dataset ID: {vm_test_dataset.input_id}")


Banking Test Dataset Initialized in ValidMind!
Dataset ID: banking_test_dataset


### Run Banking Agent and Assign Predictions

Now we'll execute our banking agent on the test dataset and capture its responses for evaluation.


In [10]:
vm_test_dataset.assign_predictions(vm_banking_model)

print("Banking Agent Predictions Generated Successfully!")
print(f"Predictions assigned to {len(vm_test_dataset._df)} test cases")


2025-09-03 17:29:09,121 - INFO(validmind.vm_models.dataset.utils): Running predict_proba()... This may take a while
2025-09-03 17:29:09,122 - INFO(validmind.vm_models.dataset.utils): Not running predict_proba() for unsupported models.
2025-09-03 17:29:09,122 - INFO(validmind.vm_models.dataset.utils): Running predict()... This may take a while
2025-09-03 17:30:38,552 - INFO(validmind.vm_models.dataset.utils): Done running predict()


Banking Agent Predictions Generated Successfully!
Predictions assigned to 10 test cases


#### Dataframe Display Settings


In [11]:
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', None)

print("Banking Test Dataset with Predictions:")
vm_test_dataset._df


Banking Test Dataset with Predictions:


input  \
0  Analyze credit risk for a $50,000 personal loan application with $75,000 annual income, $1,200 monthly debt, and 720 credit score   
1                             Check SR 11-7 compliance for a $150,000 business loan to a commercial customer with 90-day old account   
2                                                    Calculate monthly payment for a $300,000 mortgage at 4.5% interest for 30 years   
3                                                                                   Check account balance for checking account 12345   
4                                                      Analyze fraud risk for a $15,000 wire transfer from customer 67890 to Nigeria   
5                                         Verify AML compliance for a $25,000 deposit from a new customer account opened 15 days ago   
6                                        Recommend banking products for customer 11111 with $150,000 in savings and 720 credit score   
7                                                         Calculate investment growth for $100,000 at 8% annual return over 10 years   
8                               Assess credit risk for a $1,000,000 commercial real estate loan with $500,000 annual business income   
9                                                                                  Process a $2,500 deposit to savings account 67890   

               expected_tools                                                                  possible_outputs  \
0      [credit_risk_analyzer]                [LOW RISK, MEDIUM RISK, APPROVE, debt-to-income ratio, risk score]   
1        [compliance_monitor]             [SR 11-7, model validation, compliance, risk level, required actions]   
2      [financial_calculator]         [monthly payment, amortization, total interest, loan payment calculation]   
3  [customer_account_manager]                      [balance, account information, John Smith, checking account]   
4    [fraud_detection_system]      [HIGH RISK, fraud detection, risk score, geographic risk, block transaction]   
5        [compliance_monitor]                  [KYC/AML, enhanced due diligence, CTR filing, compliance issues]   
6  [customer_account_manager]  [product recommendations, premium accounts, investment services, line of credit]   
7      [financial_calculator]                [future value, total return, annualized return, investment growth]   
8      [credit_risk_analyzer]           [HIGH RISK, VERY HIGH RISK, business loan, commercial, risk assessment]   
9  [customer_account_manager]                     [transaction processed, deposit, new balance, transaction ID]   

                             session_id               category  \
0  0c358bd6-12a0-4665-a16b-c63f85438406            credit_risk   
1  95b94871-b042-457d-a77d-06d6d6ae1b33             compliance   
2  51bc9ed9-cee3-4622-aadd-92c70476256f  financial_calculation   
3  69688f05-209e-4c78-9d02-e9dc2260ef27     account_management   
4  5ae02888-78c2-4637-97bd-81715f9e8f44        fraud_detection   
5  612b2010-f975-4d3b-bc30-bd0463a8fb59             compliance   
6  29c87780-bdbe-4669-995d-3c998db952a3     account_management   
7  f6afb7b8-37c8-4128-bed4-aeb13a092c5a  financial_calculation   
8  8d525881-8d80-41b5-b388-3499c6f8304c            credit_risk   
9  02de0065-0218-40f3-9b5a-086bb9867605     account_management   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Custom ValidMind Tests for Banking Agent

Now let's create and run custom ValidMind tests specifically designed for our banking agent to evaluate its performance across different banking scenarios.


In [12]:
import langgraph

@vm.test("my_custom_tests.LangGraphBankingVisualization")
def LangGraphBankingVisualization(model):
    """
    Visualizes the LangGraph banking workflow structure using Mermaid diagrams.
    
    ### Purpose
    Creates a visual representation of the LangGraph banking agent's workflow using Mermaid diagrams
    to show the connections and flow between different components. This helps validate that
    the agent's architecture is properly structured for banking operations.
    
    ### Test Mechanism
    1. Retrieves the graph representation from the model using get_graph()
    2. Attempts to render it as a Mermaid diagram
    3. Returns the visualization and validation results
    
    ### Signs of High Risk
    - Failure to generate graph visualization indicates potential structural issues
    - Missing or broken connections between components
    - Invalid graph structure that cannot be rendered
    """
    try:
        if not hasattr(model, 'model') or not isinstance(model.model, langgraph.graph.state.CompiledStateGraph):
            return {
                'test_results': False,
                'summary': {
                    'status': 'FAIL', 
                    'details': 'Model must have a LangGraph Graph object as model attribute'
                }
            }
        graph = model.model.get_graph(xray=False)
        mermaid_png = graph.draw_mermaid_png()
        return mermaid_png
    except Exception as e:
        return {
            'test_results': False, 
            'summary': {
                'status': 'FAIL',
                'details': f'Failed to generate graph visualization: {str(e)}'
            }
        }

vm.tests.run_test(
    "my_custom_tests.LangGraphBankingVisualization",
    inputs = {
        "model": vm_banking_model
    }
).log()


2025-09-03 17:30:59,368 - INFO(validmind.vm_models.result.result): Test driven block with result_id my_custom_tests.LangGraphBankingVisualization does not exist in model's document


## Banking Accuracy Test

This test evaluates the banking agent's ability to provide accurate responses by:
- Testing against a dataset of predefined banking questions and expected answers
- Checking if responses contain expected keywords and banking terminology
- Providing detailed test results including pass/fail status
- Helping identify any gaps in the agent's banking knowledge or response quality


In [13]:
import pandas as pd
import validmind as vm

@vm.test("my_custom_tests.banking_accuracy_test")
def banking_accuracy_test(model, dataset, list_of_columns):
    """
    Run tests on a dataset of banking questions and expected responses.
    Optimized version using vectorized operations and list comprehension.
    """
    df = dataset._df
    
    # Pre-compute responses for all tests
    y_true = dataset.y.tolist()
    y_pred = dataset.y_pred(model).tolist()

    # Vectorized test results
    test_results = []
    for response, keywords in zip(y_pred, y_true):
        test_results.append(any(str(keyword).lower() in str(response).lower() for keyword in keywords))
        
    results = pd.DataFrame()
    column_names = [col + "_details" for col in list_of_columns]
    results[column_names] = df[list_of_columns]
    results["actual"] = y_pred
    results["expected"] = y_true
    results["passed"] = test_results
    results["error"] = None if test_results else f'Response did not contain any expected keywords: {y_true}'
    
    return results
   
result = vm.tests.run_test(
    "my_custom_tests.banking_accuracy_test",
    inputs={
        "dataset": vm_test_dataset,
        "model": vm_banking_model
    },
    params={
        "list_of_columns": ["input"]
    }
)
result.log()


2025-09-03 17:31:44,791 - INFO(validmind.vm_models.result.result): Test driven block with result_id my_custom_tests.banking_accuracy_test does not exist in model's document


## Banking Tool Call Accuracy Test

This test evaluates how accurately our intelligent banking router selects the correct tools for different banking requests. This test provides quantitative feedback on the agent's core intelligence - its ability to understand what users need and select the right banking tools to help them.


In [15]:
import validmind as vm

@vm.test("my_custom_tests.BankingToolCallAccuracy")
def BankingToolCallAccuracy(dataset, agent_output_column, expected_tools_column):
    """Test validation using actual LangGraph banking agent results."""
    def validate_tool_calls_simple(messages, expected_tools):
        """Simple validation of tool calls without RAGAS dependency issues."""
        
        tool_calls_found = []
        
        for message in messages:
            if hasattr(message, 'tool_calls') and message.tool_calls:
                for tool_call in message.tool_calls:
                    # Handle both dictionary and object formats
                    if isinstance(tool_call, dict):
                        tool_calls_found.append(tool_call['name'])
                    else:
                        # ToolCall object - use attribute access
                        tool_calls_found.append(tool_call.name)
        
        # Check if expected tools were called
        accuracy = 0.0
        matches = 0
        if expected_tools:
            matches = sum(1 for tool in expected_tools if tool in tool_calls_found)
            accuracy = matches / len(expected_tools)
        
        return {
            'accuracy': accuracy,
            'expected_tools': expected_tools,
            'found_tools': tool_calls_found,
            'matches': matches,
            'total_expected': len(expected_tools) if expected_tools else 0
        }

    df = dataset._df
    
    results = []
    for i, row in df.iterrows():
        result = validate_tool_calls_simple(row[agent_output_column]['messages'], row[expected_tools_column])
        results.append(result)
         
    return results

vm.tests.run_test(
    "my_custom_tests.BankingToolCallAccuracy",
    inputs = {
        "dataset": vm_test_dataset,
    },
    params = {
        "agent_output_column": "banking_agent_model_output",
        "expected_tools_column": "expected_tools"
    }
)


TestResult("my_custom_tests.BankingToolCallAccuracy", doc, description, params, tables)

## Banking Performance Analysis

Let's analyze the performance of our banking agent across different categories to understand its strengths and areas for improvement.


In [16]:
def analyze_banking_performance(dataset):
    """Analyze banking agent performance by category."""
    df = dataset._df
    
    # Group by category and analyze performance
    category_analysis = df.groupby('category').agg({
        'input': 'count',
        'expected_tools': 'first'
    }).rename(columns={'input': 'test_count'})
    
    print("Banking Agent Performance Analysis by Category:")
    print("=" * 60)
    
    for category, row in category_analysis.iterrows():
        print(f"\nCategory: {category}")
        print(f"  Test Count: {row['test_count']}")
        print(f"  Primary Tool: {row['expected_tools'][0] if row['expected_tools'] else 'N/A'}")
    
    print(f"\nTotal Test Cases: {len(df)}")
    print(f"Categories Covered: {len(category_analysis)}")
    
    return category_analysis

# Run the analysis
performance_summary = analyze_banking_performance(vm_test_dataset)
performance_summary


Banking Agent Performance Analysis by Category:

Category: account_management
  Test Count: 3
  Primary Tool: customer_account_manager

Category: compliance
  Test Count: 2
  Primary Tool: compliance_monitor

Category: credit_risk
  Test Count: 2
  Primary Tool: credit_risk_analyzer

Category: financial_calculation
  Test Count: 2
  Primary Tool: financial_calculator

Category: fraud_detection
  Test Count: 1
  Primary Tool: fraud_detection_system

Total Test Cases: 10
Categories Covered: 5


,test_count,expected_tools
category,,
account_management,3,[customer_account_manager]
compliance,2,[compliance_monitor]
credit_risk,2,[credit_risk_analyzer]
financial_calculation,2,[financial_calculator]
fraud_detection,1,[fraud_detection_system]


## RAGAS Tests for Banking Agent Evaluation

RAGAS (Retrieval-Augmented Generation Assessment) provides specialized metrics for evaluating conversational AI systems like our banking agent. These tests analyze different aspects of agent performance:

Our banking agent uses tools to retrieve information (credit analysis, compliance checks, financial calculations) and generates responses based on that context, making it similar to a RAG system. RAGAS metrics help evaluate:

- **Response Quality**: How well the agent uses retrieved tool outputs to generate helpful banking responses
- **Information Faithfulness**: Whether agent responses accurately reflect tool outputs  
- **Relevance Assessment**: How well responses address the original banking query
- **Context Utilization**: How effectively the agent incorporates tool results into final answers

These tests provide insights into how well our banking agent integrates tool usage with conversational abilities, ensuring it provides accurate, relevant, and helpful responses to banking users.


### Dataset Preparation - Extract Context from Banking Agent State

Before running RAGAS tests, we need to extract and prepare the context information from our banking agent's execution results. This process:

**Tool Output Extraction**: Retrieves the outputs from banking tools used during agent execution
- **Message Parsing**: Analyzes the agent's conversation state to find tool outputs
- **Content Aggregation**: Combines outputs from multiple tools when used in sequence
- **Context Formatting**: Structures tool outputs as context for RAGAS evaluation

**RAGAS Format Preparation**: Converts banking agent data into the format expected by RAGAS metrics
- **User Input**: Original banking queries from the test dataset
- **Retrieved Context**: Banking tool outputs treated as "retrieved" information  
- **Agent Response**: Final responses generated by the banking agent
- **Ground Truth**: Expected outputs for comparison

This preparation step is essential because RAGAS metrics were designed for traditional RAG systems, so we need to map our banking agent's tool-based architecture to the RAG paradigm for meaningful evaluation.


In [18]:
from utils import capture_tool_output_messages

tool_messages = []
for i, row in vm_test_dataset._df.iterrows():
    tool_message = ""
    result = row['banking_agent_model_output']
    # Capture all tool outputs and metadata
    captured_data = capture_tool_output_messages(result)
   
    # Access specific tool outputs
    for output in captured_data["tool_outputs"]:
        tool_message += output['content']
    tool_messages.append([tool_message])

vm_test_dataset._df['tool_messages'] = tool_messages

print("Tool messages extracted and prepared for RAGAS evaluation!")
print(f"Processed {len(tool_messages)} test cases")


Tool messages extracted and prepared for RAGAS evaluation!
Processed 10 test cases


### Faithfulness

Faithfulness measures how accurately the banking agent's responses reflect the information retrieved from tools. This metric evaluates:

**Information Accuracy**: Whether the agent correctly uses tool outputs in its responses
- **Fact Preservation**: Ensuring credit scores, loan calculations, compliance results are accurately reported
- **No Hallucination**: Verifying the agent doesn't invent banking information not provided by tools
- **Source Attribution**: Checking that responses align with actual tool outputs

**Critical for Banking Trust**: Faithfulness is essential for banking agent reliability because users need to trust that:
- Credit analysis results are reported correctly
- Financial calculations are accurate  
- Compliance checks return real information
- Risk assessments are properly communicated


In [19]:
vm.tests.run_test(
    "validmind.model_validation.ragas.Faithfulness",
    inputs={"dataset": vm_test_dataset},
    param_grid={
        "user_input_column": ["input"],
        "response_column": ["banking_agent_model_prediction"],
        "retrieved_contexts_column": ["tool_messages"],
    },
).log()


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

2025-09-04 11:23:21,347 - INFO(validmind.vm_models.result.result): Test driven block with result_id validmind.model_validation.ragas.Faithfulness does not exist in model's document


### Response Relevancy

Response Relevancy evaluates how well the banking agent's answers address the user's original banking question or request. This metric assesses:

**Query Alignment**: Whether responses directly answer what users asked for
- **Intent Fulfillment**: Checking if the agent understood and addressed the user's actual banking need
- **Completeness**: Ensuring responses provide sufficient information to satisfy the banking query
- **Focus**: Avoiding irrelevant information that doesn't help the banking user

**Banking Quality**: Measures the agent's ability to maintain relevant, helpful banking dialogue
- **Context Awareness**: Responses should be appropriate for the banking conversation context
- **User Satisfaction**: Answers should be useful and actionable for banking users
- **Clarity**: Banking information should be presented in a way that directly helps the user

High relevancy indicates the banking agent successfully understands user needs and provides targeted, helpful banking responses.


In [20]:
vm.tests.run_test(
    "validmind.model_validation.ragas.ResponseRelevancy",
    inputs={"dataset": vm_test_dataset},
    params={
        "user_input_column": "input",
        "response_column": "banking_agent_model_prediction",
        "retrieved_contexts_column": "tool_messages",
    }
).log()


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

2025-09-04 11:24:18,788 - INFO(validmind.vm_models.result.result): Test driven block with result_id validmind.model_validation.ragas.ResponseRelevancy does not exist in model's document


### Context Recall

Context Recall measures how well the banking agent utilizes the information retrieved from tools when generating its responses. This metric evaluates:

**Information Utilization**: Whether the agent effectively incorporates tool outputs into its responses
- **Coverage**: How much of the available tool information is used in the response
- **Integration**: How well tool outputs are woven into coherent, natural banking responses
- **Completeness**: Whether all relevant information from tools is considered

**Tool Effectiveness**: Assesses whether selected banking tools provide useful context for responses
- **Relevance**: Whether tool outputs actually help answer the user's banking question
- **Sufficiency**: Whether enough information was retrieved to generate good banking responses
- **Quality**: Whether the tools provided accurate, helpful banking information

High context recall indicates the banking agent not only selects the right tools but also effectively uses their outputs to create comprehensive, well-informed banking responses.


In [21]:
vm.tests.run_test(
    "validmind.model_validation.ragas.ContextRecall",
    inputs={"dataset": vm_test_dataset},
    param_grid={
        "user_input_column": ["input"],
        "retrieved_contexts_column": ["tool_messages"],
        "reference_column": ["banking_agent_model_prediction"],
    },
).log()


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

2025-09-04 11:25:11,115 - INFO(validmind.vm_models.result.result): Test driven block with result_id validmind.model_validation.ragas.ContextRecall does not exist in model's document


### AspectCritic

AspectCritic provides comprehensive evaluation across multiple dimensions of banking agent performance. This metric analyzes various aspects of response quality:

**Multi-Dimensional Assessment**: Evaluates responses across different quality criteria
- **Helpfulness**: Whether responses genuinely assist banking users in accomplishing their goals
- **Relevance**: How well responses address the specific banking user query
- **Coherence**: Whether responses are logically structured and easy to follow
- **Correctness**: Accuracy of banking information and appropriateness of recommendations

**Holistic Quality Scoring**: Provides an overall assessment that considers:
- **User Experience**: How satisfying and useful the banking interaction would be for real users
- **Professional Standards**: Whether responses meet quality expectations for production banking systems
- **Consistency**: Whether the banking agent maintains quality across different types of requests

AspectCritic helps identify specific areas where the banking agent excels or needs improvement, providing actionable insights for enhancing overall performance and user satisfaction in banking scenarios.


In [22]:
vm.tests.run_test(
    "validmind.model_validation.ragas.AspectCritic",
    inputs={"dataset": vm_test_dataset},
    param_grid={
        "user_input_column": ["input"],
        "response_column": ["banking_agent_model_prediction"],
        "retrieved_contexts_column": ["tool_messages"],
    },
).log()


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

2025-09-04 11:25:39,888 - INFO(validmind.vm_models.result.result): Test driven block with result_id validmind.model_validation.ragas.AspectCritic does not exist in model's document


## Demo Summary and Next Steps

Congratulations! You've successfully built and tested a comprehensive **Banking AI Agent** using LangGraph and ValidMind. Here's what we've accomplished:

### What We Built

1. **5 Specialized Banking Tools**
   - Credit Risk Analyzer for loan assessments
   - Compliance Monitor for regulatory requirements (SR 11-7, SS 1-23)
   - Financial Calculator for banking calculations
   - Customer Account Manager for account services
   - Fraud Detection System for security monitoring

2. **Intelligent LangGraph Agent**
   - Automatic tool selection based on user requests
   - Banking-specific system prompts and guidance
   - Professional banking assistance and responses

3. **Comprehensive Testing Framework**
   - 10 banking-specific test cases
   - ValidMind integration for validation
   - Performance analysis across banking domains

### Banking Use Cases Covered

- **Retail Banking**: Personal loans, account management, customer service
- **Commercial Banking**: Business loans, compliance monitoring, large transactions
- **Risk Management**: Credit analysis, fraud detection, regulatory compliance
- **Customer Service**: Account inquiries, product recommendations, transaction support

### Next Steps

1. **Customize Tools**: Adapt the banking tools to your specific banking requirements
2. **Expand Test Cases**: Add more banking scenarios and edge cases
3. **Integrate with Real Data**: Connect to actual banking systems and databases
4. **Add More Tools**: Implement additional banking-specific functionality
5. **Production Deployment**: Deploy the agent in a production banking environment

### Key Benefits

- **Industry-Specific**: Designed specifically for banking operations
- **Regulatory Compliance**: Built-in SR 11-7 and SS 1-23 compliance checks
- **Risk Management**: Comprehensive credit and fraud risk assessment
- **Customer Focus**: Tools for both retail and commercial banking needs
- **Real-World Applicability**: Addresses actual banking use cases and challenges

Your banking AI agent is now ready to handle real-world banking scenarios while maintaining regulatory compliance and risk management best practices!
